## Import Releveant Libraries 

In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Loading Data

In [3]:
# Loading data as 2 tuple structure 
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

## Pre Proccessing 

In [4]:
## Creating train and test variable
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

#creating validation samples using 10 percent of the training
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

#creating test samples
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#creating a function that scales input
def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [6]:
#Setting buffer size to 10000 and shuffling scaled training samples
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#creating validation dataset
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


#Creating batch size
BATCH_SIZE = 100

#setting batch to 100 to indicate to the model how many samples to take in each bacth
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

## Model

## Outline the model

In [12]:
#Creting input,output and hidden layer size
input_size = 784
out_size = 10
hidden_layer_size = 100

#Defining the model and stacking the layers
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(out_size, activation='softmax'),
    
])

## Choose the optimizer and the loss function


In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training 

In [14]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3360 - accuracy: 0.9037 - val_loss: 0.1881 - val_accuracy: 0.9483
Epoch 2/5
540/540 - 3s - loss: 0.1367 - accuracy: 0.9595 - val_loss: 0.1260 - val_accuracy: 0.9662
Epoch 3/5
540/540 - 3s - loss: 0.0961 - accuracy: 0.9714 - val_loss: 0.0977 - val_accuracy: 0.9703
Epoch 4/5
540/540 - 3s - loss: 0.0721 - accuracy: 0.9781 - val_loss: 0.0732 - val_accuracy: 0.9783
Epoch 5/5
540/540 - 3s - loss: 0.0586 - accuracy: 0.9819 - val_loss: 0.0640 - val_accuracy: 0.9805


## Test the model

In [16]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 8ms/step - loss: 0.0800 - accuracy: 0.9755
